In [112]:
import spacy
import nltk
from pprint import pprint
import random
nltk.download("all")

nlp = spacy.load("en_core_web_sm")


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /home/miles/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to /home/miles/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/miles/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nltk_data]    | Downloading package cess_esp to
[nltk_data]   

In [113]:
import os
def load_corpus(folder):
    corpus = []
    for root, dirs, files in os.walk(folder, topdown=False):
        for name in files:
            try:
                with open(os.path.join(root, name), "r") as fp:
                    corpus.append(fp.read())
            except UnicodeDecodeError as e:
                print(e.__str__(), "for", os.path.join(root, name))
    return corpus

def load_corpuses(folder):
    sub_folders = []
    for root, dirs, files in os.walk(folder):
        if dirs:
            for dir_ in dirs:
                sub_folders.append(dir_)

    corpuses = {}
    print(sub_folders)
    for sub_folder in sub_folders:
        print(os.path.join(folder, sub_folder))
        corpuses[sub_folder] = load_corpus(os.path.join(folder, sub_folder))
    return corpuses

corpuses = load_corpuses("bbc")

['business', 'tech', 'entertainment', 'sport', 'politics']
bbc/business
bbc/tech
bbc/entertainment
bbc/sport
'utf-8' codec can't decode byte 0xa3 in position 257: invalid start byte for bbc/sport/199.txt
bbc/politics


In [114]:
for corpus in corpuses:
    for story in corpuses[corpus][:10]:
        for ent in nlp(story).ents:
            print(ent.text, ent.start_char, ent.end_char, ent.label_)

Diageo 0 6 PERSON
US 14 16 GPE
Diageo 28 34 PERSON
Californian 91 102 ORG
Chalone 116 123 GPE
260 129 132 MONEY
134 136 139 MONEY
Diageo 173 179 PERSON
Smirnoff 208 216 ORG
Guinness 227 235 ORG
US 260 262 GPE
Diageo 311 317 PERSON
US 341 343 GPE
the first quarter of 2005 384 409 DATE
Chalone 419 426 PERSON
US 465 467 GPE
US 489 491 GPE
Diageo 540 546 PERSON
Diageo North America 606 626 PERSON
Ivan Menezes 637 649 PERSON
July 654 658 DATE
Diageo 660 666 PERSON
the London Stock Exchange 687 712 ORG
annual 726 732 DATE
8.89bn 746 752 MONEY
9.28bn 765 771 MONEY
a year earlier 772 786 DATE
the year ending 31 December 2003 841 873 DATE
Chalone 875 882 PERSON
69.4 905 909 MONEY
Man Utd 17 24 WORK_OF_ART
Malcolm Glazer 35 49 PERSON
Manchester United 83 100 GPE
Premiership 140 151 ORG
800 161 164 MONEY
US 172 174 GPE
the last 12 months 216 234 DATE
United 255 261 ORG
Glazer 316 322 PERSON
the Tampa Bay Buccaneers 333 357 ORG
Glazer 482 488 PERSON
one 505 508 CARDINAL
the Red Devils 541 555 ORG


In [115]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords

vectorizer = CountVectorizer(stop_words=stopwords.words('english'))

all_ = []
for corpus in corpuses:
    all_ += corpuses[corpus]

vectorizer.fit(all_)
# x = vectorizer.transform(corpuses["tech"])
# vectorizer.vocabulary_.get('Sony')

CountVectorizer(stop_words=['i', 'me', 'my', 'myself', 'we', 'our', 'ours',
                            'ourselves', 'you', "you're", "you've", "you'll",
                            "you'd", 'your', 'yours', 'yourself', 'yourselves',
                            'he', 'him', 'his', 'himself', 'she', "she's",
                            'her', 'hers', 'herself', 'it', "it's", 'its',
                            'itself', ...])

In [116]:
vectorizer.transform(['regulatory approval']).toarray()[0]


array([0, 0, 0, ..., 0, 0, 0])

In [117]:
import math
x = []
y =[]
for corpus in corpuses:
    for story in corpuses[corpus]:
        x.append(vectorizer.transform([story]).toarray()[0])
        y.append(corpus)
pprint(x[:3])
pprint(y[:3])

[array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0]),
 array([0, 0, 0, ..., 0, 0, 0])]
['business', 'business', 'business']


In [118]:
def y_to_int(y):
    # Move to sklearn.preprocessing.LabelEncoder
    result = []
    for value in y:
        if value == "business":
            result.append(1)
        elif value == "entertainment":
            result.append(2)
        elif value == "tech":
            result.append(3)
        elif value == "sport":
            result.append(4)
        elif value == "politics":
            result.append(5)
        else:
            result.append(0)
    return result

In [122]:
size_dataset_full=len(x)
size_test=int(round(size_dataset_full*0.2,0))

list_test_indices=random.sample(range(size_dataset_full), size_test)

test_x = []
test_y = []
train_x = []
train_y = []

for i,example in enumerate(x):
  if i in list_test_indices:
      test_x.append(example)
      test_y.append(y[i])
  else:
      train_x.append(example)
      train_y.append(y[i])

test_y = y_to_int(test_y)
train_y = y_to_int(train_y)

In [103]:
print(test_y)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [110]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm

svm_clf=make_pipeline(StandardScaler(), svm.SVC(cache_size=2000, decision_function_shape='ovo'))

In [121]:
svm_clf.fit(train_x, train_y)


ValueError: Expected 2D array, got 1D array instead:
array=[].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.